In [ ]:
#Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Read the csv files
olympics_df = pd.read_csv('Resources/athlete_events.csv')
arrivals_df = pd.read_csv('Resources/INT.ARVL.csv')
departures_df = pd.read_csv('Resources/INT.DPRT.csv')
expenditures_df = pd.read_csv('Resources/INT.XPND.csv')

In [ ]:
olympics_df.head()

In [ ]:
arrivals_df.head()

In [ ]:
departures_df.head() 

In [ ]:
expenditures_df.head()